In [1]:
import itertools
import shutil
from multiprocessing import Pool
from src.recommenders import ImprovedRecommender, PopBasedRecommender
from os import cpu_count
from os.path import exists
from sklearn.decomposition import TruncatedSVD
qual_eval_folder = './evaluation'

From the base recommender results, we will now continue to use the ideal combination of methods as a baseline before we make further improvements.
This means we use cosine distance and normalized feature vectors. We also don't use feedback weighting.

The following cell runs the basic recommender and provides more in-depth evaluation and compares it to a popularity-based recommender.

In [6]:
rec = PopBasedRecommender()
rec.generate_recommendations("./data/australian_user_reviews.json", read_max=1000)
print(rec.evaluate())
dim_red = TruncatedSVD(n_components=200)
rec = ImprovedRecommender("./data/steam_games.json", sparse=True, tfidf='smooth', use_feedback=False, normalize=True, dim_red=dim_red)
rec.generate_recommendations("./data/australian_user_reviews.json", read_max=1000)
rec.evaluate()

1000it [00:00, 19590.03it/s]


Break reading after 1000 records
Reading 1001 rows.
{'nDCG@k': 0.506053017999911, 'recall@k': 0.06972603153421748, 'ideal_recall@k': 0.1745280595968364, 'nRecall@k': 0.493580313359646}


32135it [00:01, 21770.31it/s]


Reading 32135 rows.


1000it [00:00, 19589.94it/s]


Break reading after 1000 records
Reading 1001 rows.


c:\Users\Sam\Documents\GitHub\AI-Project\src\recommenders.py:405: RuntimeWarning: Sparse was set to 'True' but dimensionality reduction is used, using dense matrix representation instead.
  warnings.warn("Sparse was set to 'True' but dimensionality reduction is used, using dense matrix representation instead.", RuntimeWarning)
999it [00:38, 26.06it/s]


{'nDCG@k': 0.18435874971021057,
 'recall@k': 0.021854696803229632,
 'ideal_recall@k': 0.17286358361933224,
 'nRecall@k': 0.17024927971940373}